In [29]:
import facebook
from datetime import datetime, timedelta
import pandas as pd

In [96]:
access_token = "EAACEdEose0cBAC4gsR2VCsk5ZBtDBIv9FGTrAZCixsmSZBLq5kBn33ASEbvHf5e6Vxed9RLx4m5vvmIhIAwWbqXUnbZCdNV7FzhQVUcpkp80UklIPOdka1bgb8yfPeZBeeuUYbre1K4jmrkuhSauXi5l1eRcGTIVWBipLeyYNciUHNz0EgkB8wbqHZC7gQyLAZD"
graph = facebook.GraphAPI(access_token, version="2.10")

In [46]:
"""Retrive post_id of a page which posted with chopped_days"""
def retrive_postsid(page_id, chopped_days):
    page_posts = graph.get_object('/'+str(page_id)+'/posts?')
    page_posts_used = []
    post_items = []
    while True:
        for post in page_posts['data']:
            created_time = datetime.strptime(post['created_time'][:19], '%Y-%m-%dT%H:%M:%S')
            chopped_time = datetime.now() - timedelta(days=chopped_days)
            if created_time>chopped_time:
                post_item = dict()
                post_item['post_id'] = post['id']
                post_item['page_id'] = page_id
                post_item['created_time'] = post['created_time'][:10]
                post_items.append(post_item)
                page_posts_used.append(post['id'])
            else:
                break
        if created_time<chopped_time:
            break
        if 'next' in page_posts['paging'].keys():
            after_key = page_posts['paging']['cursors']['after']
            page_posts = graph.get_object('/'+str(page_id)+'/posts?after='+after_key)
        else:
            break
    
    return page_posts_used, pd.DataFrame(post_items)

In [91]:
def retrive_reactions(posts_id, page_id):
    reaction_items = []
    i = 1
    for post_id in posts_id:
        print('retriving reactions')
        print(i, '/', len(posts_id) ,'At post_id: ', post_id, )
        i = i + 1
        post_reactions = graph.get_object('/'+str(post_id)+'/reactions?limit=200')
        if bool(post_reactions['data']):    
            while True:
                for reaction in post_reactions['data']:
                    reaction_item = dict()
                    reaction_item['user_id'] = reaction['id']
                    reaction_item['post_id'] = post_id
                    reaction_item['page_id'] = page_id
                    reaction_item['reaction_type'] = reaction['type']
                    reaction_items.append(reaction_item)
                if 'next' in post_reactions['paging']:
                    after_key = post_reactions['paging']['cursors']['after']
                    post_reactions = graph.get_object('/'+str(post_id)+'/reactions?after='+str(after_key))
                else:
                    break
    print('finish retriving reactions!')
    return pd.DataFrame(reaction_items)  

In [92]:
def retrive_comments(posts_id, page_id):    
    comment_items = []
    i=1
    for post_id in posts_id:
        print('retriving comments')
        print(i, '/', len(posts_id) ,'At post_id: ', post_id, )
        i = i + 1
        post_comments = graph.get_object('/'+str(post_id)+'/comments?limit=200')
        if bool(post_comments['data']):
            while True:
                for comment in post_comments['data']:
                    comment_item = dict()
                    comment_item['user_id'] = comment['from']['id']
                    comment_item['post_id'] = post_id
                    comment_item['page_id'] = '366743047016721'
                    comment_item['comment_body'] = comment['message']
                    comment_items.append(comment_item)
                if 'next' in post_comments['paging']:
                    after_key = post_comments['paging']['cursors']['after']
                    post_comments = graph.get_object('/'+str(post_id)+'/comments?after='+str(after_key))
                else:
                    break
    print('finish retriving comments!')
    return pd.DataFrame(comment_items) 

In [49]:
def retrive_pageid(page_name):
    return graph.get_object('/'+str(page_name))['id']

In [77]:
def retrive_data(page_name, chopped_days):
    from pathlib import Path
    comments_dir = Path('/data/comments/'+page_name+'.csv')
    page_id = retrive_pageid(page_name)
    posts_id, db_posts = retrive_postsid(page_id, chopped_days)
    db_posts.to_csv('./data/posts/'+page_name+'.csv')
    if comments_dir.exists():
    else:
        db_comments = retrive_comments(posts_id, page_id)
        db_comments.to_csv('./data/comments/'+page_name+'.csv')
    db_reactions = retrive_reactions(posts_id, page_id)
    db_reactions.to_csv('./data/reactions/'+page_name+'.csv') 
    return db_comments, db_reactions, db_posts

IndentationError: expected an indented block (<ipython-input-77-9838ea89276c>, line 8)

In [54]:
db_comments = dict()
db_reactions = dict()
db_posts = dict()
db_comments['snapbeforeeatofficial'], db_reactions['snapbeforeeatofficial'], db_posts['snapbeforeeatofficial'] = retrive_data('snapbeforeeatofficial', 180)

1 / 151 At post_id:  464171817085120_857626517739646
2 / 151 At post_id:  464171817085120_857217481113883
3 / 151 At post_id:  464171817085120_855312391304392
4 / 151 At post_id:  464171817085120_854846234684341
5 / 151 At post_id:  464171817085120_853973438104954
6 / 151 At post_id:  464171817085120_853913604777604
7 / 151 At post_id:  464171817085120_853245311511100
8 / 151 At post_id:  464171817085120_852982684870696
9 / 151 At post_id:  464171817085120_852786564890308
10 / 151 At post_id:  464171817085120_852600378242260
11 / 151 At post_id:  464171817085120_852063581629273
12 / 151 At post_id:  464171817085120_851823724986592
13 / 151 At post_id:  464171817085120_851394875029477
14 / 151 At post_id:  464171817085120_851205585048406
15 / 151 At post_id:  464171817085120_850958201739811
16 / 151 At post_id:  464171817085120_850459585123006
17 / 151 At post_id:  464171817085120_850016721833959
18 / 151 At post_id:  464171817085120_849187315250233
19 / 151 At post_id:  464171817085120

finish!
1 / 151 At post_id:  464171817085120_857626517739646
2 / 151 At post_id:  464171817085120_857217481113883
3 / 151 At post_id:  464171817085120_855312391304392
4 / 151 At post_id:  464171817085120_854846234684341
5 / 151 At post_id:  464171817085120_853973438104954
6 / 151 At post_id:  464171817085120_853913604777604
7 / 151 At post_id:  464171817085120_853245311511100
8 / 151 At post_id:  464171817085120_852982684870696
9 / 151 At post_id:  464171817085120_852786564890308
10 / 151 At post_id:  464171817085120_852600378242260
11 / 151 At post_id:  464171817085120_852063581629273
12 / 151 At post_id:  464171817085120_851823724986592
13 / 151 At post_id:  464171817085120_851394875029477
14 / 151 At post_id:  464171817085120_851205585048406
15 / 151 At post_id:  464171817085120_850958201739811
16 / 151 At post_id:  464171817085120_850459585123006
17 / 151 At post_id:  464171817085120_850016721833959
18 / 151 At post_id:  464171817085120_849187315250233
19 / 151 At post_id:  4641718

finish!


In [60]:
db_posts['snapbeforeeatofficial'].describe()

,created_time,page_id,post_id
count,151,151,151
unique,112,1,151
top,2017-07-12,464171817085120,464171817085120_850016721833959
freq,4,151,1


In [61]:
db_comments['snapbeforeeatofficial'].describe()

,comment_body,page_id,post_id,user_id
count,3942,3942,3942,3942
unique,3805,1,83,1919
top,,366743047016721,464171817085120_837407669761531,942031282514104
freq,32,3942,589,49


In [62]:
db_reactions['snapbeforeeatofficial'].describe()

,page_id,post_id,reaction_type,user_id
count,11593,11593,11593,11593
unique,1,150,7,6256
top,464171817085120,464171817085120_773596672809298,LIKE,830242230372817
freq,11593,2968,11281,129


In [66]:
db_comments['snapbeforeeatofficial'].to_csv('./data/comments/snapbeforeeatoffical.csv')

In [67]:
db_posts['snapbeforeeatofficial'].to_csv('./data/posts/snapbeforeeatoffical.csv')

In [68]:
db_reactions['snapbeforeeatofficial'].to_csv('./data/reactions/snapbeforeeatoffical.csv')

In [70]:
db_comments['dinewithpigs'], db_reactions['dinewithpigs'], db_posts['dinewithpigs'] = retrive_data('dinewithpigs', 180)

1 / 46 At post_id:  960782227349574_1535027806591677
2 / 46 At post_id:  960782227349574_1528070270620764
3 / 46 At post_id:  960782227349574_1526998074061317
4 / 46 At post_id:  960782227349574_1522545277839930
5 / 46 At post_id:  960782227349574_1522013361226455
6 / 46 At post_id:  960782227349574_1516823638412094
7 / 46 At post_id:  960782227349574_1513862062041585
8 / 46 At post_id:  960782227349574_1508167732611018
9 / 46 At post_id:  960782227349574_1505587889535669
10 / 46 At post_id:  960782227349574_1504718872955904
11 / 46 At post_id:  960782227349574_1502308386530286
12 / 46 At post_id:  960782227349574_1502252766535848
13 / 46 At post_id:  960782227349574_1500616230032835
14 / 46 At post_id:  960782227349574_1498787256882399
15 / 46 At post_id:  960782227349574_1494749160619542
16 / 46 At post_id:  960782227349574_1490927821001676
17 / 46 At post_id:  960782227349574_1487269911367467
18 / 46 At post_id:  960782227349574_1482505035177288
19 / 46 At post_id:  960782227349574_

In [76]:
db_comments['paidonnnn'], db_reactions['paidonnnn'], db_posts['paidonnnn'] = retrive_data('paidonnnn', 180)

retriving comments
1 / 406 At post_id:  1436860363229411_1934197283495714
retriving comments
2 / 406 At post_id:  1436860363229411_1934104793504963
retriving comments
3 / 406 At post_id:  1436860363229411_1934095193505923
retriving comments
4 / 406 At post_id:  1436860363229411_1934083340173775
retriving comments
5 / 406 At post_id:  1436860363229411_1933139913601451
retriving comments
6 / 406 At post_id:  1436860363229411_1933055403609902
retriving comments
7 / 406 At post_id:  1436860363229411_1932917160290393
retriving comments
8 / 406 At post_id:  1436860363229411_1932752270306882
retriving comments
9 / 406 At post_id:  1436860363229411_1932685773646865
retriving comments
10 / 406 At post_id:  1436860363229411_1931941553721287
retriving comments
11 / 406 At post_id:  1436860363229411_1931522567096519
retriving comments
12 / 406 At post_id:  1436860363229411_1931570983758344
retriving comments
13 / 406 At post_id:  1436860363229411_1931516123763830
retriving comments
14 / 406 At pos

retriving comments
111 / 406 At post_id:  1436860363229411_1911246309124145
retriving comments
112 / 406 At post_id:  1436860363229411_1910863245829118
retriving comments
113 / 406 At post_id:  1436860363229411_1910670595848383
retriving comments
114 / 406 At post_id:  1436860363229411_1910517679197008
retriving comments
115 / 406 At post_id:  1436860363229411_1910518792530230
retriving comments
116 / 406 At post_id:  1436860363229411_1909301452651964
retriving comments
117 / 406 At post_id:  1436860363229411_1910093615906081
retriving comments
118 / 406 At post_id:  1436860363229411_1910083205907122
retriving comments
119 / 406 At post_id:  1436860363229411_1909872962594813
retriving comments
120 / 406 At post_id:  1436860363229411_1909083512673758
retriving comments
121 / 406 At post_id:  1436860363229411_1909722205943222
retriving comments
122 / 406 At post_id:  1436860363229411_1909476479301128
retriving comments
123 / 406 At post_id:  1436860363229411_1908897632692346
retriving co

retriving comments
219 / 406 At post_id:  1436860363229411_1896500023932107
retriving comments
220 / 406 At post_id:  1436860363229411_1896155213966588
retriving comments
221 / 406 At post_id:  1436860363229411_1895713310677445
retriving comments
222 / 406 At post_id:  1436860363229411_1896145787300864
retriving comments
223 / 406 At post_id:  1436860363229411_1896123843969725
retriving comments
224 / 406 At post_id:  1436860363229411_1896077787307664
retriving comments
225 / 406 At post_id:  1436860363229411_1895475580701218
retriving comments
226 / 406 At post_id:  1436860363229411_1895677310681045
retriving comments
227 / 406 At post_id:  1436860363229411_1895243617391081
retriving comments
228 / 406 At post_id:  1436860363229411_1892273724354737
retriving comments
229 / 406 At post_id:  1436860363229411_1894835830765193
retriving comments
230 / 406 At post_id:  1436860363229411_1895276467387796
retriving comments
231 / 406 At post_id:  1436860363229411_1894851637430279
retriving co

retriving comments
327 / 406 At post_id:  1436860363229411_1880139372234839
retriving comments
328 / 406 At post_id:  1436860363229411_1880128465569263
retriving comments
329 / 406 At post_id:  1436860363229411_1879773148938128
retriving comments
330 / 406 At post_id:  1436860363229411_1879322122316564
retriving comments
331 / 406 At post_id:  1436860363229411_1878899495692160
retriving comments
332 / 406 At post_id:  1436860363229411_1878822099033233
retriving comments
333 / 406 At post_id:  1436860363229411_1864301520485291
retriving comments
334 / 406 At post_id:  1436860363229411_1877454502503326
retriving comments
335 / 406 At post_id:  1436860363229411_1877011182547658
retriving comments
336 / 406 At post_id:  1436860363229411_1876163155965794
retriving comments
337 / 406 At post_id:  1436860363229411_1874718379443605
retriving comments
338 / 406 At post_id:  1436860363229411_1874633236118786
retriving comments
339 / 406 At post_id:  1436860363229411_1873963822852394
retriving co

retriving reactions
29 / 406 At post_id:  1436860363229411_1928764230705686
retriving reactions
30 / 406 At post_id:  1436860363229411_1928604457388330
retriving reactions
31 / 406 At post_id:  1436860363229411_1928521784063264
retriving reactions
32 / 406 At post_id:  1436860363229411_1928515624063880
retriving reactions
33 / 406 At post_id:  1436860363229411_1928211800760929
retriving reactions
34 / 406 At post_id:  1436860363229411_1928505510731558
retriving reactions
35 / 406 At post_id:  1436860363229411_1928386117410164
retriving reactions
36 / 406 At post_id:  1436860363229411_1928062790775830
retriving reactions
37 / 406 At post_id:  1436860363229411_1928102314105211
retriving reactions
38 / 406 At post_id:  1436860363229411_1927841100797999
retriving reactions
39 / 406 At post_id:  1436860363229411_1927839767464799
retriving reactions
40 / 406 At post_id:  1436860363229411_1927791877469588
retriving reactions
41 / 406 At post_id:  1436860363229411_1927699020812207
retriving re

retriving reactions
137 / 406 At post_id:  1436860363229411_1906620342920075
retriving reactions
138 / 406 At post_id:  1436860363229411_1906273386288104
retriving reactions
139 / 406 At post_id:  1436860363229411_1905560126359430
retriving reactions
140 / 406 At post_id:  1436860363229411_1905477946367648
retriving reactions
141 / 406 At post_id:  1436860363229411_1905096266405816
retriving reactions
142 / 406 At post_id:  1436860363229411_1904457863136323
retriving reactions
143 / 406 At post_id:  1436860363229411_1904746509774125
retriving reactions
144 / 406 At post_id:  1436860363229411_1904651899783586
retriving reactions
145 / 406 At post_id:  1436860363229411_1904447613137348
retriving reactions
146 / 406 At post_id:  1436860363229411_1904395786475864
retriving reactions
147 / 406 At post_id:  1436860363229411_1904311403150969
retriving reactions
148 / 406 At post_id:  1436860363229411_1904260783156031
retriving reactions
149 / 406 At post_id:  1436860363229411_1904240136491429

retriving reactions
244 / 406 At post_id:  1436860363229411_1893968727518570
retriving reactions
245 / 406 At post_id:  1436860363229411_1893187000930076
retriving reactions
246 / 406 At post_id:  1436860363229411_1893829120865864
retriving reactions
247 / 406 At post_id:  1436860363229411_1893841577531285
retriving reactions
248 / 406 At post_id:  1436860363229411_1892699274312182
retriving reactions
249 / 406 At post_id:  1436860363229411_1893160407599402
retriving reactions
250 / 406 At post_id:  1436860363229411_1893427524239357
retriving reactions
251 / 406 At post_id:  1436860363229411_1892460471002729
retriving reactions
252 / 406 At post_id:  1436860363229411_1893116480937128
retriving reactions
253 / 406 At post_id:  1436860363229411_1893042777611165
retriving reactions
254 / 406 At post_id:  1436860363229411_1892918140956962
retriving reactions
255 / 406 At post_id:  1436860363229411_1892052834376826
retriving reactions
256 / 406 At post_id:  1436860363229411_1892698047645638

retriving reactions
351 / 406 At post_id:  1436860363229411_1871235163125260
retriving reactions
352 / 406 At post_id:  1436860363229411_1870884269827016
retriving reactions
353 / 406 At post_id:  1436860363229411_1870401393208637
retriving reactions
354 / 406 At post_id:  1436860363229411_1870312853217491
retriving reactions
355 / 406 At post_id:  1436860363229411_1870246439890799
retriving reactions
356 / 406 At post_id:  1436860363229411_1870023756579734
retriving reactions
357 / 406 At post_id:  1436860363229411_1869556963293080
retriving reactions
358 / 406 At post_id:  1436860363229411_1869243199991123
retriving reactions
359 / 406 At post_id:  1436860363229411_1869226503326126
retriving reactions
360 / 406 At post_id:  1436860363229411_1868750213373755
retriving reactions
361 / 406 At post_id:  1436860363229411_1868376920077751
retriving reactions
362 / 406 At post_id:  1436860363229411_1867965786785531
retriving reactions
363 / 406 At post_id:  1436860363229411_1867854486796661

In [78]:
db_comments['bangkokeatbig'], db_reactions['bangkokeatbig'], db_posts['bangkokeatbig'] = retrive_data('bangkokeatbig', 180)

retriving comments
1 / 92 At post_id:  218892821565212_1417237535064062
retriving comments
2 / 92 At post_id:  218892821565212_1417237375064078
retriving comments
3 / 92 At post_id:  218892821565212_1413971108724038
retriving comments
4 / 92 At post_id:  218892821565212_1413131872141295
retriving comments
5 / 92 At post_id:  218892821565212_1410231182431364
retriving comments
6 / 92 At post_id:  218892821565212_1406608562793626
retriving comments
7 / 92 At post_id:  218892821565212_1406718339449315
retriving comments
8 / 92 At post_id:  218892821565212_1405794399541709
retriving comments
9 / 92 At post_id:  218892821565212_1400213230099826
retriving comments
10 / 92 At post_id:  218892821565212_1399550303499452
retriving comments
11 / 92 At post_id:  218892821565212_1397976453656837
retriving comments
12 / 92 At post_id:  218892821565212_1392614767526339
retriving comments
13 / 92 At post_id:  218892821565212_1390909881030161
retriving comments
14 / 92 At post_id:  218892821565212_1385

retriving reactions
21 / 92 At post_id:  218892821565212_1373741849413631
retriving reactions
22 / 92 At post_id:  218892821565212_1370301596424323
retriving reactions
23 / 92 At post_id:  218892821565212_1367594486695034
retriving reactions
24 / 92 At post_id:  218892821565212_1366034810184335
retriving reactions
25 / 92 At post_id:  218892821565212_1366541160133700
retriving reactions
26 / 92 At post_id:  218892821565212_1365970420190774
retriving reactions
27 / 92 At post_id:  218892821565212_1363470177107465
retriving reactions
28 / 92 At post_id:  218892821565212_1351875361600280
retriving reactions
29 / 92 At post_id:  218892821565212_1349808961806920
retriving reactions
30 / 92 At post_id:  218892821565212_1348255265295623
retriving reactions
31 / 92 At post_id:  218892821565212_1347506758703807
retriving reactions
32 / 92 At post_id:  218892821565212_1332246596896490
retriving reactions
33 / 92 At post_id:  218892821565212_1334298366691313
retriving reactions
34 / 92 At post_id

In [79]:
db_comments['bornmadeak'], db_reactions['bornmadeak'], db_posts['bornmadeak'] = retrive_data('bornmadeak', 180)

retriving comments
1 / 78 At post_id:  427521930787483_712824862257187
retriving comments
2 / 78 At post_id:  427521930787483_711255732414100
retriving comments
3 / 78 At post_id:  427521930787483_710086859197654
retriving comments
4 / 78 At post_id:  427521930787483_709801539226186
retriving comments
5 / 78 At post_id:  427521930787483_708040972735576
retriving comments
6 / 78 At post_id:  427521930787483_706593646213642
retriving comments
7 / 78 At post_id:  427521930787483_704649706408036
retriving comments
8 / 78 At post_id:  427521930787483_703849036488103
retriving comments
9 / 78 At post_id:  427521930787483_703416639864676
retriving comments
10 / 78 At post_id:  427521930787483_702341979972142
retriving comments
11 / 78 At post_id:  427521930787483_699009273638746
retriving comments
12 / 78 At post_id:  427521930787483_698753106997696
retriving comments
13 / 78 At post_id:  427521930787483_698320823707591
retriving comments
14 / 78 At post_id:  427521930787483_696839120522428
r

retriving reactions
37 / 78 At post_id:  427521930787483_677062352500105
retriving reactions
38 / 78 At post_id:  427521930787483_675627415976932
retriving reactions
39 / 78 At post_id:  427521930787483_674267292779611
retriving reactions
40 / 78 At post_id:  427521930787483_673647922841548
retriving reactions
41 / 78 At post_id:  427521930787483_672547376284936
retriving reactions
42 / 78 At post_id:  427521930787483_667804480092559
retriving reactions
43 / 78 At post_id:  427521930787483_667777493428591
retriving reactions
44 / 78 At post_id:  427521930787483_667292823477058
retriving reactions
45 / 78 At post_id:  427521930787483_662347673971573
retriving reactions
46 / 78 At post_id:  427521930787483_661422160730791
retriving reactions
47 / 78 At post_id:  427521930787483_661263167413357
retriving reactions
48 / 78 At post_id:  427521930787483_660014834204857
retriving reactions
49 / 78 At post_id:  427521930787483_659535430919464
retriving reactions
50 / 78 At post_id:  4275219307

In [80]:
db_comments['DakNaeNon'], db_reactions['DakNaeNon'], db_posts['DakNaeNon'] = retrive_data('DakNaeNon', 180)

retriving comments
1 / 188 At post_id:  551268918372423_882695395229772
retriving comments
2 / 188 At post_id:  551268918372423_882637748568870
retriving comments
3 / 188 At post_id:  551268918372423_881579165341395
retriving comments
4 / 188 At post_id:  551268918372423_878433242322654
retriving comments
5 / 188 At post_id:  551268918372423_878024919030153
retriving comments
6 / 188 At post_id:  551268918372423_877369145762397
retriving comments
7 / 188 At post_id:  551268918372423_877040772461901
retriving comments
8 / 188 At post_id:  551268918372423_875697922596186
retriving comments
9 / 188 At post_id:  551268918372423_875664022599576
retriving comments
10 / 188 At post_id:  551268918372423_875520309280614
retriving comments
11 / 188 At post_id:  551268918372423_875510175948294
retriving comments
12 / 188 At post_id:  551268918372423_875371472628831
retriving comments
13 / 188 At post_id:  551268918372423_875348239297821
retriving comments
14 / 188 At post_id:  551268918372423_874

retriving comments
114 / 188 At post_id:  551268918372423_824913987674580
retriving comments
115 / 188 At post_id:  551268918372423_824804277685551
retriving comments
116 / 188 At post_id:  551268918372423_824165854416060
retriving comments
117 / 188 At post_id:  551268918372423_823785101120802
retriving comments
118 / 188 At post_id:  551268918372423_823480444484601
retriving comments
119 / 188 At post_id:  551268918372423_822533551245957
retriving comments
120 / 188 At post_id:  551268918372423_822411054591540
retriving comments
121 / 188 At post_id:  551268918372423_822376494594996
retriving comments
122 / 188 At post_id:  551268918372423_821394844693161
retriving comments
123 / 188 At post_id:  551268918372423_820544371444875
retriving comments
124 / 188 At post_id:  551268918372423_820368734795772
retriving comments
125 / 188 At post_id:  551268918372423_820281244804521
retriving comments
126 / 188 At post_id:  551268918372423_820228601476452
retriving comments
127 / 188 At post_i

retriving reactions
37 / 188 At post_id:  551268918372423_866103560222289
retriving reactions
38 / 188 At post_id:  551268918372423_865384996960812
retriving reactions
39 / 188 At post_id:  551268918372423_865351806964131
retriving reactions
40 / 188 At post_id:  551268918372423_865279903637988
retriving reactions
41 / 188 At post_id:  551268918372423_865156176983694
retriving reactions
42 / 188 At post_id:  551268918372423_864111557088156
retriving reactions
43 / 188 At post_id:  551268918372423_862095937289718
retriving reactions
44 / 188 At post_id:  551268918372423_861667087332603
retriving reactions
45 / 188 At post_id:  551268918372423_861656450667000
retriving reactions
46 / 188 At post_id:  551268918372423_861543980678247
retriving reactions
47 / 188 At post_id:  551268918372423_860751314090847
retriving reactions
48 / 188 At post_id:  551268918372423_859677997531512
retriving reactions
49 / 188 At post_id:  551268918372423_858379900994655
retriving reactions
50 / 188 At post_i

retriving reactions
148 / 188 At post_id:  551268918372423_813578312141481
retriving reactions
149 / 188 At post_id:  551268918372423_813562268809752
retriving reactions
150 / 188 At post_id:  551268918372423_812733928892586
retriving reactions
151 / 188 At post_id:  551268918372423_812078272291485
retriving reactions
152 / 188 At post_id:  551268918372423_811098622389450
retriving reactions
153 / 188 At post_id:  551268918372423_811065319059447
retriving reactions
154 / 188 At post_id:  551268918372423_809631092536203
retriving reactions
155 / 188 At post_id:  551268918372423_809515982547714
retriving reactions
156 / 188 At post_id:  551268918372423_809169832582329
retriving reactions
157 / 188 At post_id:  551268918372423_808555815977064
retriving reactions
158 / 188 At post_id:  551268918372423_808168816015764
retriving reactions
159 / 188 At post_id:  551268918372423_808125362686776
retriving reactions
160 / 188 At post_id:  551268918372423_807711816061464
retriving reactions
161 /

In [85]:
db_comments['ipedinth'], db_reactions['ipedinth'], db_posts['ipedinth'] = retrive_data('ipedinth', 180)

retriving comments
1 / 177 At post_id:  369540316568629_756224051233585
retriving comments
2 / 177 At post_id:  369540316568629_755435714645752
retriving comments
3 / 177 At post_id:  369540316568629_754239391432051
retriving comments
4 / 177 At post_id:  369540316568629_754049961450994
retriving comments
5 / 177 At post_id:  369540316568629_753473221508668
retriving comments
6 / 177 At post_id:  369540316568629_751401135049210
retriving comments
7 / 177 At post_id:  369540316568629_750063271849663
retriving comments
8 / 177 At post_id:  369540316568629_749662741889716
retriving comments
9 / 177 At post_id:  369540316568629_748463008676356
retriving comments
10 / 177 At post_id:  369540316568629_748037148718942
retriving comments
11 / 177 At post_id:  369540316568629_747572508765406
retriving comments
12 / 177 At post_id:  369540316568629_747212438801413
retriving comments
13 / 177 At post_id:  369540316568629_746763072179683
retriving comments
14 / 177 At post_id:  369540316568629_746

retriving comments
114 / 177 At post_id:  369540316568629_708675982655059
retriving comments
115 / 177 At post_id:  369540316568629_708194242703233
retriving comments
116 / 177 At post_id:  369540316568629_707510756104915
retriving comments
117 / 177 At post_id:  369540316568629_707486866107304
retriving comments
118 / 177 At post_id:  369540316568629_706418236214167
retriving comments
119 / 177 At post_id:  369540316568629_705291206326870
retriving comments
120 / 177 At post_id:  369540316568629_704873836368607
retriving comments
121 / 177 At post_id:  369540316568629_704324176423573
retriving comments
122 / 177 At post_id:  369540316568629_703770113145646
retriving comments
123 / 177 At post_id:  369540316568629_702401429949181
retriving comments
124 / 177 At post_id:  369540316568629_701913919997932
retriving comments
125 / 177 At post_id:  369540316568629_701441476711843
retriving comments
126 / 177 At post_id:  369540316568629_701030376752953
retriving comments
127 / 177 At post_i

retriving reactions
48 / 177 At post_id:  369540316568629_734368620085795
retriving reactions
49 / 177 At post_id:  369540316568629_734108613445129
retriving reactions
50 / 177 At post_id:  369540316568629_733718426817481
retriving reactions
51 / 177 At post_id:  369540316568629_732749733581017
retriving reactions
52 / 177 At post_id:  369540316568629_732373840285273
retriving reactions
53 / 177 At post_id:  369540316568629_731842707005053
retriving reactions
54 / 177 At post_id:  369540316568629_731419650380692
retriving reactions
55 / 177 At post_id:  369540316568629_730975260425131
retriving reactions
56 / 177 At post_id:  369540316568629_730397450482912
retriving reactions
57 / 177 At post_id:  369540316568629_730330620489595
retriving reactions
58 / 177 At post_id:  369540316568629_729523460570311
retriving reactions
59 / 177 At post_id:  369540316568629_728596453996345
retriving reactions
60 / 177 At post_id:  369540316568629_728063124049678
retriving reactions
61 / 177 At post_i

retriving reactions
158 / 177 At post_id:  369540316568629_685839664938691
retriving reactions
159 / 177 At post_id:  369540316568629_685230038332987
retriving reactions
160 / 177 At post_id:  369540316568629_684910545031603
retriving reactions
161 / 177 At post_id:  369540316568629_684461561743168
retriving reactions
162 / 177 At post_id:  369540316568629_684373475085310
retriving reactions
163 / 177 At post_id:  369540316568629_684103881778936
retriving reactions
164 / 177 At post_id:  369540316568629_683077875214870
retriving reactions
165 / 177 At post_id:  369540316568629_682781821911142
retriving reactions
166 / 177 At post_id:  369540316568629_682261931963131
retriving reactions
167 / 177 At post_id:  369540316568629_681923201997004
retriving reactions
168 / 177 At post_id:  369540316568629_681342362055088
retriving reactions
169 / 177 At post_id:  369540316568629_681020272087297
retriving reactions
170 / 177 At post_id:  369540316568629_680540142135310
retriving reactions
171 /

In [90]:
db_comments['AroiTongShare'], db_reactions['AroiTongShare'], db_posts['AroiTongShare'] = retrive_data('AroiTongShare', 180)

retriving comments
1 / 136 At post_id:  778290312277419_1289799427793169
retriving comments
2 / 136 At post_id:  778290312277419_1289799227793189
retriving comments
3 / 136 At post_id:  778290312277419_1288934641212981
retriving comments
4 / 136 At post_id:  778290312277419_1288932881213157
retriving comments
5 / 136 At post_id:  778290312277419_1288079347965177
retriving comments
6 / 136 At post_id:  778290312277419_1287936551312790
retriving comments
7 / 136 At post_id:  778290312277419_1287906467982465
retriving comments
8 / 136 At post_id:  778290312277419_1287869994652779
retriving comments
9 / 136 At post_id:  778290312277419_1283335255106253
retriving comments
10 / 136 At post_id:  778290312277419_1286218708151241
retriving comments
11 / 136 At post_id:  778290312277419_1285779441528501
retriving comments
12 / 136 At post_id:  778290312277419_1283129978460114
retriving comments
13 / 136 At post_id:  778290312277419_1283139005125878
retriving comments
14 / 136 At post_id:  778290

retriving comments
112 / 136 At post_id:  778290312277419_1161970087242771
retriving comments
113 / 136 At post_id:  778290312277419_1161765197263260
retriving comments
114 / 136 At post_id:  778290312277419_1160922874014159
retriving comments
115 / 136 At post_id:  778290312277419_1160544804051966
retriving comments
116 / 136 At post_id:  778290312277419_1159477644158682
retriving comments
117 / 136 At post_id:  778290312277419_1158529810920132
retriving comments
118 / 136 At post_id:  778290312277419_1154636064642840
retriving comments
119 / 136 At post_id:  778290312277419_1154463371326776
retriving comments
120 / 136 At post_id:  778290312277419_1153659724740474
retriving comments
121 / 136 At post_id:  778290312277419_1153561808083599
retriving comments
122 / 136 At post_id:  778290312277419_1152860264820420
retriving comments
123 / 136 At post_id:  778290312277419_1152760484830398
retriving comments
124 / 136 At post_id:  778290312277419_1152677594838687
retriving comments
125 / 

retriving reactions
85 / 136 At post_id:  778290312277419_1194229067350206
retriving reactions
86 / 136 At post_id:  778290312277419_1193590980747348
retriving reactions
87 / 136 At post_id:  778290312277419_1193341910772255
retriving reactions
88 / 136 At post_id:  778290312277419_1192272957545817
retriving reactions
89 / 136 At post_id:  778290312277419_1191268057646307
retriving reactions
90 / 136 At post_id:  778290312277419_1190484654391314
retriving reactions
91 / 136 At post_id:  778290312277419_1189231137849999
retriving reactions
92 / 136 At post_id:  778290312277419_1186974718075641
retriving reactions
93 / 136 At post_id:  778290312277419_1186004898172623
retriving reactions
94 / 136 At post_id:  778290312277419_1186092408163872
retriving reactions
95 / 136 At post_id:  778290312277419_1184470504992729
retriving reactions
96 / 136 At post_id:  778290312277419_1180846398688473
retriving reactions
97 / 136 At post_id:  778290312277419_1181079538665159
retriving reactions
98 / 

In [94]:
db_comments['224201151372283'], db_reactions['224201151372283'], db_posts['224201151372283'] = retrive_data('224201151372283', 180)

retriving comments
1 / 486 At post_id:  224201151372283_361246164334447
retriving comments
2 / 486 At post_id:  224201151372283_360024597789937
retriving comments
3 / 486 At post_id:  224201151372283_360010087791388
retriving comments
4 / 486 At post_id:  224201151372283_360005441125186
retriving comments
5 / 486 At post_id:  224201151372283_360002197792177
retriving comments
6 / 486 At post_id:  224201151372283_360000697792327
retriving comments
7 / 486 At post_id:  224201151372283_359219567870440
retriving comments
8 / 486 At post_id:  224201151372283_358875087904888
retriving comments
9 / 486 At post_id:  224201151372283_358868951238835
retriving comments
10 / 486 At post_id:  224201151372283_358868371238893
retriving comments
11 / 486 At post_id:  224201151372283_358867957905601
retriving comments
12 / 486 At post_id:  224201151372283_358853864573677
retriving comments
13 / 486 At post_id:  224201151372283_358525591273171
retriving comments
14 / 486 At post_id:  224201151372283_358

retriving comments
114 / 486 At post_id:  224201151372283_346109272514803
retriving comments
115 / 486 At post_id:  224201151372283_346052202520510
retriving comments
116 / 486 At post_id:  224201151372283_346043452521385
retriving comments
117 / 486 At post_id:  224201151372283_345998785859185
retriving comments
118 / 486 At post_id:  224201151372283_344888082636922
retriving comments
119 / 486 At post_id:  224201151372283_344900792635651
retriving comments
120 / 486 At post_id:  224201151372283_344896679302729
retriving comments
121 / 486 At post_id:  224201151372283_344891739303223
retriving comments
122 / 486 At post_id:  224201151372283_344887459303651
retriving comments
123 / 486 At post_id:  224201151372283_344597149332682
retriving comments
124 / 486 At post_id:  224201151372283_344535516005512
retriving comments
125 / 486 At post_id:  224201151372283_343814536077610
retriving comments
126 / 486 At post_id:  224201151372283_343768239415573
retriving comments
127 / 486 At post_i

retriving comments
225 / 486 At post_id:  224201151372283_331179327341131
retriving comments
226 / 486 At post_id:  224201151372283_330879254037805
retriving comments
227 / 486 At post_id:  224201151372283_330812150711182
retriving comments
228 / 486 At post_id:  224201151372283_330811854044545
retriving comments
229 / 486 At post_id:  224201151372283_330527030739694
retriving comments
230 / 486 At post_id:  224201151372283_330515334074197
retriving comments
231 / 486 At post_id:  224201151372283_330508117408252
retriving comments
232 / 486 At post_id:  224201151372283_330499750742422
retriving comments
233 / 486 At post_id:  224201151372283_330493907409673
retriving comments
234 / 486 At post_id:  224201151372283_330068730785524
retriving comments
235 / 486 At post_id:  224201151372283_330068510785546
retriving comments
236 / 486 At post_id:  224201151372283_330068314118899
retriving comments
237 / 486 At post_id:  224201151372283_330068094118921
retriving comments
238 / 486 At post_i

retriving comments
336 / 486 At post_id:  224201151372283_316284162163981
retriving comments
337 / 486 At post_id:  224201151372283_316229115502819
retriving comments
338 / 486 At post_id:  224201151372283_316115402180857
retriving comments
339 / 486 At post_id:  224201151372283_316102495515481
retriving comments
340 / 486 At post_id:  224201151372283_315748248884239
retriving comments
341 / 486 At post_id:  224201151372283_315201668938897
retriving comments
342 / 486 At post_id:  224201151372283_315171455608585
retriving comments
343 / 486 At post_id:  224201151372283_314793568979707
retriving comments
344 / 486 At post_id:  224201151372283_314792338979830
retriving comments
345 / 486 At post_id:  224201151372283_314757212316676
retriving comments
346 / 486 At post_id:  224201151372283_314327452359652
retriving comments
347 / 486 At post_id:  224201151372283_314262539032810
retriving comments
348 / 486 At post_id:  224201151372283_314241242368273
retriving comments
349 / 486 At post_i

retriving comments
447 / 486 At post_id:  224201151372283_294696150989449
retriving comments
448 / 486 At post_id:  224201151372283_294455064346891
retriving comments
449 / 486 At post_id:  224201151372283_294419927683738
retriving comments
450 / 486 At post_id:  224201151372283_294279967697734
retriving comments
451 / 486 At post_id:  224201151372283_293966944395703
retriving comments
452 / 486 At post_id:  224201151372283_293930287732702
retriving comments
453 / 486 At post_id:  224201151372283_293822837743447
retriving comments
454 / 486 At post_id:  224201151372283_293371404455257
retriving comments
455 / 486 At post_id:  224201151372283_293369504455447
retriving comments
456 / 486 At post_id:  224201151372283_293079317817799
retriving comments
457 / 486 At post_id:  224201151372283_292693121189752
retriving comments
458 / 486 At post_id:  224201151372283_292604347865296
retriving comments
459 / 486 At post_id:  224201151372283_292392181219846
retriving comments
460 / 486 At post_i

retriving reactions
72 / 486 At post_id:  224201151372283_352067025252361
retriving reactions
73 / 486 At post_id:  224201151372283_352070541918676
retriving reactions
74 / 486 At post_id:  224201151372283_352093555249708


GraphAPIError: Error validating access token: Session has expired on Saturday, 28-Oct-17 05:00:00 PDT. The current time is Saturday, 28-Oct-17 05:00:01 PDT.

In [97]:
db_comments['Dakmaipenyaadak'], db_reactions['Dakmaipenyaadak'], db_posts['Dakmaipenyaadak'] = retrive_data('Dakmaipenyaadak', 180)

retriving comments
1 / 15 At post_id:  220711858140080_640305446180717
retriving comments
2 / 15 At post_id:  220711858140080_640007959543799
retriving comments
3 / 15 At post_id:  220711858140080_635483446662917
retriving comments
4 / 15 At post_id:  220711858140080_634792100065385
retriving comments
5 / 15 At post_id:  220711858140080_634073556803906
retriving comments
6 / 15 At post_id:  220711858140080_610676915810237
retriving comments
7 / 15 At post_id:  220711858140080_609283225949606
retriving comments
8 / 15 At post_id:  220711858140080_603369776540951
retriving comments
9 / 15 At post_id:  220711858140080_595465573998038
retriving comments
10 / 15 At post_id:  220711858140080_590609431150319
retriving comments
11 / 15 At post_id:  220711858140080_585445498333379
retriving comments
12 / 15 At post_id:  220711858140080_585384648339464
retriving comments
13 / 15 At post_id:  220711858140080_585317558346173
retriving comments
14 / 15 At post_id:  220711858140080_585144931696769
r

In [98]:
db_comments['semagudak'], db_reactions['semagudak'], db_posts['semagudak'] = retrive_data('semagudak', 180)

retriving comments
1 / 75 At post_id:  1886953594900349_1944660209129687
retriving comments
2 / 75 At post_id:  1886953594900349_1944414495820925
retriving comments
3 / 75 At post_id:  1886953594900349_1943368642592177
retriving comments
4 / 75 At post_id:  1886953594900349_1943130209282687
retriving comments
5 / 75 At post_id:  1886953594900349_1942915682637473
retriving comments
6 / 75 At post_id:  1886953594900349_1942836449312063
retriving comments
7 / 75 At post_id:  1886953594900349_1942550832673958
retriving comments
8 / 75 At post_id:  1886953594900349_1942431802685861
retriving comments
9 / 75 At post_id:  1886953594900349_1942135449382163
retriving comments
10 / 75 At post_id:  1886953594900349_1941363726126002
retriving comments
11 / 75 At post_id:  1886953594900349_1940988136163561
retriving comments
12 / 75 At post_id:  1886953594900349_1940623456200029
retriving comments
13 / 75 At post_id:  1886953594900349_1939859059609802
retriving comments
14 / 75 At post_id:  1886953

retriving reactions
37 / 75 At post_id:  1886953594900349_1922387651356943
retriving reactions
38 / 75 At post_id:  1886953594900349_1922080621387646
retriving reactions
39 / 75 At post_id:  1886953594900349_1921543221441386
retriving reactions
40 / 75 At post_id:  1886953594900349_1920321314896910
retriving reactions
41 / 75 At post_id:  1886953594900349_1919688034960238
retriving reactions
42 / 75 At post_id:  1886953594900349_1919220415007000
retriving reactions
43 / 75 At post_id:  1886953594900349_1917350748527300
retriving reactions
44 / 75 At post_id:  1886953594900349_1916504371945271
retriving reactions
45 / 75 At post_id:  1886953594900349_1914376038824771
retriving reactions
46 / 75 At post_id:  1886953594900349_1913519478910427
retriving reactions
47 / 75 At post_id:  1886953594900349_1912774178984957
retriving reactions
48 / 75 At post_id:  1886953594900349_1912760645652977
retriving reactions
49 / 75 At post_id:  1886953594900349_1911817059080669
retriving reactions
50 / 